In [3]:
import sys; sys.path.insert(0,'..')
from bardbot import *

In [40]:
def generate(
        s,
        max_new_tokens=20,
        top_k=50, 
        top_p=0.95,
        **kwargs
    ):
    tokenizer = get_tokenizer()
    inputs = tokenizer(s, return_tensors="pt")
    
    model=get_model()
    generation_output = model.generate(
        **inputs,
        do_sample=True, 
        # max_length=max_length, 
        top_k=top_k, 
        top_p=top_p, 
        num_return_sequences=1,
        max_new_tokens=max_new_tokens,
        no_repeat_ngram_size=2
    )
    o=tokenizer.decode(generation_output[0])
    return o[len(s):]
    #o=[tokenizer.decode(x) for x in generation_output[0]]
    # return o

In [50]:
%%timeit
s='From fairest creatures we desire increase'
# generate(s)

6.15 ns ± 0.0345 ns per loop (mean ± std. dev. of 7 runs, 100,000,000 loops each)


In [42]:
import prosodic as p
p.config['print_to_screen']=0

In [43]:
t=p.Text(s)

In [53]:
s1=s
s2='But as the riper should by time decease,'
l1=p.Text(s1).lines()[0]
l2=p.Text(s2).lines()[0]
print(l1)

From fairest creatures we desire increase


In [45]:
l1.rime_distance(l2)

0.0

In [71]:
def get_num_sylls(w):
    w=w.strip()
    if not w: return 0
    if not any(x.isalpha() for x in w): return 0
    return p.Word(w).num_syll

def get_lineobj(line):
    return p.Text(line).lines()[0] if type(line)==str else line

def rhymes(line1,line2,max_rhyme_dist=5):
    line1=get_lineobj(line1)
    line2=get_lineobj(line2)
    word1=line1.words()[-1].token
    word2=line2.words()[-1].token
    if word1==word2: return False
    dist=line1.rime_distance(line2)
    # print(dist,line1,'|',line2)
    return dist<=max_rhyme_dist

def ends_on_stress(line):
    lineobj=get_lineobj(line)
    sylls=lineobj.syllables()
    return sylls[-1].feats['prom.stress']>0

In [74]:
def gen_poem(
        prompt="Let us",
        num_lines=14,
        num_sylls=10,
        rhyme_scheme='abab cdcd efef gg',
        max_rhyme_dist=3):
    lines=[]
    rhyme_scheme=rhyme_scheme.replace(' ','')
    while len(lines)<num_lines:
        line_i=len(lines)
        txt='\n'.join(lines) if lines else prompt
        line = generate(txt,max_new_tokens=20).replace('\n',' ')
        if not lines: line=prompt+line
        toks = line.strip().split()
        ns=0
        linetoks=[]
        for tok in toks:
            nstok=get_num_sylls(tok)
            if ns+nstok>num_sylls: break
            linetoks.append(tok)
            ns+=nstok
        if ns!=num_sylls: continue
        # if linetoks[-1].isalpha(): continue
        
        oline=' '.join(linetoks).strip()
        # print('?',oline)
        lineobj = p.Text(oline).lines()[0]
        
        if not ends_on_stress(lineobj): continue
        
        if lines:
            line_rhyme_letter=rhyme_scheme[line_i]
            line_i_matcher=rhyme_scheme.index(line_rhyme_letter)
            if line_i_matcher<line_i:
                oline0 = lines[line_i_matcher]
                lineobj0 = p.Text(oline0).lines()[0]
                if not rhymes(lineobj, lineobj0, max_rhyme_dist=max_rhyme_dist): continue
    
        print(oline)
        lines.append(oline)

In [76]:
gen_poem(prompt='Look in thy glass and',num_lines=4,max_rhyme_dist=2)

Look in thy glass and cast a spell, and then,
after thou shalt receive them, return home;
In any case, all my life I have been
